In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt

# recommended styles: 'bmh' 'fivethirtyeight' 'ggplot' 'seaborn-darkgrid'
plt.style.use("bmh")
plt.rcParams["axes.axisbelow"] = True

In [ ]:
import numpy as np
import pandas
import pandas as pd
from pandas import DataFrame, Index, Series, Timedelta, Timestamp


from tsdm.utils.strings import snake2camel

rng = np.random.default_rng()
np.set_printoptions()

## Helper Functions

In [ ]:
def update_legend(ax, legend_dict, *, legend_kwargs=None, **kwargs):
    """Update the legend with a dict[line, "name"]. Useful for dynamically updating the legend."""
    _legend_kwargs = dict(
        frameon=False,
        loc="upper right",
        bbox_to_anchor=(0.75, 0.95, 0.05, 0.05),
        mode="expand",
    )
    legend_kwargs = _legend_kwargs.update(
        {} if legend_kwargs is None else legend_kwargs
    )
    for key, val in kwargs.items():
        legend_dict[key] = val
    leg = ax.legend(legend_dict.keys(), legend_dict.values(), **legends_kwargs)

In [ ]:
def data_overview(df: DataFrame):
    overview = DataFrame(index=df.columns)
    mask = pandas.isna(df)
    overview["# datapoints"] = (~mask).sum()
    overview["% missing"] = (mask.mean() * 100).round(2)
    overview["min"] = df.min().round(2)
    overview["mean"] = df.mean().round(2)
    overview["std"] = df.std().round(2)
    overview["max"] = df.max().round(2)
    overview["unit"] = units[df.columns]
    freq = {}
    for col in df:
        mask = pandas.notna(df[col])
        time = pandas.to_timedelta(df["runtime"][mask], unit="h")
        freq[col] = time.diff().mean()
    overview["freq"] = Series(freq).round("s").dt.to_pytimedelta().astype(str)
    return overview

## Some prep work

In [ ]:
from tsdm.datasets import KIWI_RUNS

data = KIWI_RUNS.load("measurements_aggregated")
data = data.rename(columns={col: snake2camel(col) for col in data})

for run_exp in data.reset_index(level=2).index.unique():
    time = data.loc[run_exp].index
    td = (time - time.min()) / Timedelta("1h")
    data.loc[run_exp, "runtime"] = td

data

In [ ]:
_units = data["Unit"]
data = data.drop(columns="Unit")
data = data.astype("float32")

units = Series(dtype=pandas.StringDtype())

for col in data:
    if col == "runtime":
        continue
    mask = pandas.notna(data[col])
    unit = _units[mask].unique().to_list()
    assert len(unit) <= 1, f"{col}, {unit} {len(unit)}"
    units[col] = unit[0]

units["DOT"] = "%"
units["OD600"] = "%"
units["PH"] = "-log[H⁺]"
units["Acetate"] = "g/L"
units["runtime"] = "h"
units = units.fillna(pd.NA).astype("string").astype("category")

## Overview Raw Data

In [ ]:
overview = data_overview(data.reset_index(level=[0, 1], drop=True))
with pd.option_context("display.float_format", "{:,.2f}".format):
    display(overview)

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=4, figsize=(12, 12))
data["runtime"] = data["runtime"].diff()
for col, ax in zip(data, axes.flatten()):
    vals = data[col]
    mask = pandas.notna(vals)
    ax.hist(vals[mask], bins=100, density=True)
    ax.set_title(col)
    # ax.set_xscale("symlog")
    ax.set_yscale("log")

In [ ]:
for col in ["OD600", "DOT", "Acetate", "Glucose"]:
    print(data[col][data[col] < 0])

## Overview cleaned data

In [ ]:
from tsdm.datasets import KIWI_RUNS

data = KIWI_RUNS.dataset
data = data.rename(columns={col: snake2camel(col) for col in data})

In [ ]:
for run_exp in data.reset_index(level=2).index.unique():
    time = data.loc[run_exp].index
    td = (time - time.min()) / Timedelta("1h")
    data.loc[run_exp, "runtime"] = td

overview = data_overview(data.reset_index(level=[0, 1], drop=True))
with pd.option_context("display.float_format", "{:,.2f}".format):
    display(overview)

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=4, figsize=(12, 12))

for col, ax in zip(data, axes.flatten()):
    vals = data[col]
    mask = pandas.notna(vals)
    ax.hist(vals[mask], bins=100)
    ax.set_title(col)
    ax.set_xscale("symlog")
    # ax.set_yscale("log")

## Overview Task data

pretty much the same as cleaned but without run 355

In [ ]:
from tsdm.tasks import KIWI_RUNS_TASK

data = KIWI_RUNS_TASK().timeseries
data = data.rename(columns={col: snake2camel(col) for col in data})
for run_exp in data.reset_index(level=2).index.unique():
    time = data.loc[run_exp].index
    td = (time - time.min()) / Timedelta("1h")
    data.loc[run_exp, "runtime"] = td

overview = data_overview(data.reset_index(level=[0, 1], drop=True))
with pd.option_context("display.float_format", "{:,.2f}".format):
    display(overview)

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=3, figsize=(16, 9))

for col, ax in zip(data, axes.flatten()):
    vals = data[col]
    mask = pandas.notna(vals)
    ax.hist(vals[mask], bins=50)
    ax.set_title(col)
    ax.set_xscale("symlog")
    ax.set_yscale("log")
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.grid(axis="x")